In [1]:
import keras
keras.__version__

'2.9.0'

In [13]:
%pip install pyyaml h5py

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) 
#have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.

from keras.datasets import imdb
vocabulary_size = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocabulary_size)

In [37]:
print(test_data[0])

[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 2, 38, 32, 25, 7944, 451, 202, 14, 6, 717]


In [5]:
#A review from IMDB Dataset with its label
print('---review---')
print(train_data[6])
print('---label---')
print(train_labels[6])

---review---
[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [6]:
# Since we restricted ourselves to the top 10,000 most frequent words, no word index will exceed 10,000:
max([max(sequence) for sequence in train_data])

9999

In [7]:
print('Maximum review length: {}'.format(
len(max((train_data + test_data), key=len))))

Maximum review length: 2697


In [8]:
print('Minimum review length: {}'.format(
len(min((train_data + test_data), key=len))))

Minimum review length: 70


In [9]:
# In order to feed this data into our RNN, all input documents must have the same length.
#  We will limit the maximum review length to max_words by truncating longer reviews and padding shorter reviews with a null value (0). 
# We can accomplish this using the pad_sequences() function in Keras. For now, set max_words to 500.

from keras.utils import pad_sequences
max_words = 500
X_train = pad_sequences(train_data, maxlen=max_words)
X_test = pad_sequences(test_data, maxlen=max_words)

In [10]:
#To summarize, our model is a simple RNN model with 1 embedding, 1 LSTM and 1 dense layers. 213,301 parameters in total need to be trained.

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [14]:
# X_valid & y_valid is the Testing data while X_train2 & y_train2 is the Training data

batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], train_labels[:batch_size]
X_train2, y_train2 = X_train[batch_size:], train_labels[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
model.save('my_model.h5')

Epoch 1/3
390/390 [==============================] - 446s 1s/step - loss: 0.3027 - accuracy: 0.8771 - val_loss: 0.1579 - val_accuracy: 0.9375
Epoch 2/3
390/390 [==============================] - 388s 994ms/step - loss: 0.2105 - accuracy: 0.9216 - val_loss: 0.3388 - val_accuracy: 0.8594
Epoch 3/3
390/390 [==============================] - 443s 1s/step - loss: 0.1802 - accuracy: 0.9337 - val_loss: 0.2114 - val_accuracy: 0.9219


In [15]:
scores = model.evaluate(X_test, train_labels, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.49900001287460327


In [32]:
predictions = model.predict(X_test[:2])

1/1 [==============================] - 0s 119ms/step


In [57]:
# This is not needed here, it is a practice and this section logic will be applied on user review data
#print(predictions[0])
#print(predictions[1])
def encode(str):
    import numpy as np
    wordIndex = imdb.get_word_index()
    list = []
    str = str.split()
    for words in str:
        #print(words)
        list.append(wordIndex[words])
    #print(list)
    list = np.array(list)
    return list

x = 'the film was so freaking awesome although it was very strange and directed in a noir mode'
newList = encode(x)
final_list = []
final_list.append(newList)
print(final_list)
from keras.utils import pad_sequences
max_words = 500
final_list = pad_sequences(final_list, maxlen=max_words) 
#print(final_list)
predictions = model.predict(final_list)
predictions



[array([   1,   19,   13,   35, 7110, 1187,  258,    9,   13,   52,  677,
          2,  523,    8,    3, 1356, 5437])]
1/1 [==============================] - 0s 114ms/step


array([[0.79778624]], dtype=float32)